In [3]:
# CÉLULA 1: Visor, GPS e Conexão 
import sys, pandas as pd
from pathlib import Path
import numpy as np


# Configura o visor financeiro
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.4f}'.format

# GPS Dinâmico e Blindado: Acha a raiz do projeto automaticamente
raiz = Path.cwd() if Path.cwd().joinpath('src').exists() else Path.cwd().parent
sys.path.append(str(raiz))

# Conecta os motores
from src.data_ingestion import carregar_base_basileia
from src.feature_engineering import calcular_historico_credito, padronizar_tempo_emprego, criar_variaveis_dummy, imputar_dados_nulos


print(f"STATUS: Cockpit alinhado. Rota travada em: {raiz}")

STATUS: Cockpit alinhado. Rota travada em: C:\Users\igorc\OneDrive\Área de Trabalho\basel_risk_modeling


In [4]:
# A Execução do ETL
print("STATUS: Iniciando esteira de processamento...")

# 1. Extração
loan_data = carregar_base_basileia()

# 2. Tratamento Temporal
loan_data['emp_length_int'] = padronizar_tempo_emprego(loan_data)
loan_data['mths_since_earliest_cr_line'] = calcular_historico_credito(loan_data, 'earliest_cr_line')

# 3. Tratamento de Nulos (O nosso novo motor de Imputação)
loan_data = imputar_dados_nulos(loan_data)

# 4. Criação do Gabarito (Target) e Isolamento de Vazamento de Dados
status_ruins = ['Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off', 'Late (31-120 days)']
import numpy as np # Garantindo que o motor matemático está ligado
loan_data['good_bad_loan'] = np.where(loan_data['loan_status'].isin(status_ruins), 0, 1)

# 5. Explosão de Categóricas (Dummies)
colunas_para_binario = [
    'grade', 'sub_grade', 'home_ownership', 'verification_status', 
    'purpose', 'addr_state', 'initial_list_status'
]
loan_data = criar_variaveis_dummy(loan_data, colunas_para_binario)

# 6. Limpeza de RAM (Descarte de colunas processadas)
loan_data = loan_data.drop(columns=['earliest_cr_line', 'emp_length', 'loan_status'])

print(f"STATUS: SUCESSO. Base limpa e lacrada. Formato atual: {loan_data.shape}")

STATUS: Iniciando esteira de processamento...
STATUS: Rota confirmada. Iniciando leitura de loan_data_2007_2014.csv...
SUCESSO: 466285 linhas extraídas para a RAM.
STATUS: SUCESSO. Base limpa e lacrada. Formato atual: (466285, 178)
